In [6]:
import mlflow

from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [7]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',    # Experiment ID we want
    filter_string="metrics.f1 > 0.25",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.f1 ASC"]
)

In [8]:
for run in runs:
    print(f"run id: {run.info.run_id}, f1: {run.data.metrics['f1']:.4f}")

run id: 481c870084bd4239966f5b8f1a96be2b, f1: 0.2564
run id: 9c6c79ee89254d6bb3acf21158826bfa, f1: 0.2581
run id: e8c57447e4c64a47912c1541ea649605, f1: 0.2632
run id: 7bcb7f8ad71342e5870e9ab81c037b5b, f1: 0.2632
run id: 5044013a27594bd8a5a96df96d59ee29, f1: 0.2759


In [9]:
logged_model = 'runs:/' + run.info.run_id + '/models'

print(logged_model)

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

runs:/5044013a27594bd8a5a96df96d59ee29/models


Exception: Invalid parent directory '/workspaces/Testrepo/mlruns/.trash'

In [ ]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "70cc9d23610d4dd9969187cbfabb08a1"
model_uri = f"runs:/{run_id}/models"
mlflow.register_model(model_uri=model_uri, name="MRI_predictor")